# 機械学習をPythonで実践する-18　　～ 様々なエンコーディング手法 ～

In [20]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.compose import ColumnTransformer
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from lightGBM_cv import lightGBM_classifier_cv_func
from category_encoders import TargetEncoder


%matplotlib inline
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Label Encoding
ペンギンデータセットに対してラベルエンコーディングを適用する。  
このデータセットの質的変数は順序尺度ではないので、ラベルエンコーディングは適さないが、  
決定木やGBDTであればラベルエンコーディングで問題ないので、これを想定してやってみる。

In [15]:
# データセットのカラム名やデータ型は分かっているので、dtypesを指定しておく。
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

In [14]:
df.head()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
str,str,f32,f32,f32,f32,str
"""Adelie""","""Torgersen""",39.099998,18.700001,181.0,3750.0,"""MALE"""
"""Adelie""","""Torgersen""",39.5,17.4,186.0,3800.0,"""FEMALE"""
"""Adelie""","""Torgersen""",40.299999,18.0,195.0,3250.0,"""FEMALE"""
"""Adelie""","""Torgersen""",null,null,null,null,null
"""Adelie""","""Torgersen""",36.700001,19.299999,193.0,3450.0,"""FEMALE"""


In [7]:
# 量的変数と質的変数のカラム名をそれぞれ取得。今回は目的変数も含めてエンコーディングしちゃう。
num_cols = df.select(pl.col(pl.Float32)).columns
cat_cols = df.select(pl.col(pl.Utf8)).columns

In [17]:
# ラベルエンコーディング（LabelEncoderではなく、OrdinalEncoderを使う）
oe = OrdinalEncoder()
# pandasで返ってくるように指定。polarsは指定できない模様
oe.set_output(transform='pandas')
# カテゴリ変数をエンコーディング。polars.DFはそのまま入れられないのでpandasに変換する。
df = df.with_columns( pl.from_pandas(oe.fit_transform(df.select(cat_cols).to_pandas())) )

In [18]:
df

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
f64,f64,f32,f32,f32,f32,f64
0.0,2.0,39.099998,18.700001,181.0,3750.0,2.0
0.0,2.0,39.5,17.4,186.0,3800.0,1.0
0.0,2.0,40.299999,18.0,195.0,3250.0,1.0
0.0,2.0,null,null,null,null,3.0
0.0,2.0,36.700001,19.299999,193.0,3450.0,1.0
0.0,2.0,39.299999,20.6,190.0,3650.0,2.0
0.0,2.0,38.900002,17.799999,181.0,3625.0,1.0
0.0,2.0,39.200001,19.6,195.0,4675.0,2.0
0.0,2.0,34.099998,18.1,193.0,3475.0,3.0


sexカラムにあった欠損値は3.0にラベル付けされた模様。

## Target Encoding

scikit-LearnにはTarget Encoding用のパッケージは入っていないので、  
pip installする必要あり。  

category_encoders.TargetEncoder()クラスを使う。  
使い方はscikit-learnの各クラスと同様。
* インスタンス生成
* .fit(X, y) → TargetEncodingなので、目的変数もfitに渡す必要あり。
* .transform(X)
  * Leakage防止のため、.fitの際にテストデータを含まないように注意。今回の例では全て学習データとして扱う。

In [19]:
%pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### - 二値分類の場合

In [41]:
# データ読み込み
df = pl.from_pandas(sns.load_dataset('titanic'))
df = df.drop_nulls()
df.head()

survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
i64,i64,str,f64,i64,i64,f64,str,cat,str,bool,cat,str,str,bool
1,1,"""female""",38.0,1,0,71.2833,"""C""","""First""","""woman""",false,"""C""","""Cherbourg""","""yes""",false
1,1,"""female""",35.0,1,0,53.1,"""S""","""First""","""woman""",false,"""C""","""Southampton""","""yes""",false
0,1,"""male""",54.0,0,0,51.8625,"""S""","""First""","""man""",true,"""E""","""Southampton""","""no""",true
1,3,"""female""",4.0,1,1,16.7,"""S""","""Third""","""child""",false,"""G""","""Southampton""","""yes""",false
1,1,"""female""",58.0,0,0,26.55,"""S""","""First""","""woman""",false,"""C""","""Southampton""","""yes""",true


In [31]:
# TargetEncoderのインスタンス生成。.fitの際は当然目的変数も入れる必要がある。
# fitに渡すdfはpolarsではなくpandasである必要あり。
X = df.drop(['survived', 'alive']).to_pandas()
y = df['survived'].to_pandas()
encoder = TargetEncoder()
encoder.fit(X, y)
encoder.transform(X)

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,1,0.931533,38.0,1,0,71.2833,0.737773,0.675159,0.950662,False,0.592011,0.737773,False
1,1,0.931533,35.0,1,0,53.1000,0.643481,0.675159,0.950662,False,0.592011,0.643481,False
2,1,0.436317,54.0,0,0,51.8625,0.643481,0.675159,0.391155,True,0.717867,0.643481,True
3,3,0.931533,4.0,1,1,16.7000,0.643481,0.628538,0.732342,False,0.646289,0.643481,False
4,1,0.931533,58.0,0,0,26.5500,0.643481,0.675159,0.950662,False,0.592011,0.643481,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1,0.931533,47.0,1,1,52.5542,0.643481,0.675159,0.950662,False,0.725425,0.643481,False
178,1,0.436317,33.0,0,0,5.0000,0.643481,0.675159,0.391155,True,0.737957,0.643481,True
179,1,0.931533,56.0,0,1,83.1583,0.737773,0.675159,0.950662,False,0.592011,0.737773,False
180,1,0.931533,19.0,0,0,30.0000,0.643481,0.675159,0.950662,False,0.737957,0.643481,True


カテゴリカラムはターゲットエンコーディングされたが、元々booleanだったadult_male、aloneはそのままになってしまっている。  
これらをエンコーディング対象に含めるには文字列型にキャストする必要あり。

In [44]:
df2 = df.with_columns(pl.col('adult_male').cast(pl.Utf8),
                pl.col('alone').cast(pl.Utf8))
df2.head()

survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
i64,i64,str,f64,i64,i64,f64,str,cat,str,str,cat,str,str,str
1,1,"""female""",38.0,1,0,71.2833,"""C""","""First""","""woman""","""false""","""C""","""Cherbourg""","""yes""","""false"""
1,1,"""female""",35.0,1,0,53.1,"""S""","""First""","""woman""","""false""","""C""","""Southampton""","""yes""","""false"""
0,1,"""male""",54.0,0,0,51.8625,"""S""","""First""","""man""","""true""","""E""","""Southampton""","""no""","""true"""
1,3,"""female""",4.0,1,1,16.7,"""S""","""Third""","""child""","""false""","""G""","""Southampton""","""yes""","""false"""
1,1,"""female""",58.0,0,0,26.55,"""S""","""First""","""woman""","""false""","""C""","""Southampton""","""yes""","""true"""


boolean→categoricalにキャストすると、0,1(UInt)に置き換わってしまい、  
TargetEncodingされないようなので注意。

In [45]:
X = df2.drop(['survived', 'alive']).to_pandas()
y = df2['survived'].to_pandas()
encoder = TargetEncoder()
encoder.fit(X, y)
encoder.transform(X)

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,1,0.931533,38.0,1,0,71.2833,0.737773,0.675159,0.950662,0.936698,0.592011,0.737773,0.721144
1,1,0.931533,35.0,1,0,53.1000,0.643481,0.675159,0.950662,0.936698,0.592011,0.643481,0.721144
2,1,0.436317,54.0,0,0,51.8625,0.643481,0.675159,0.391155,0.391155,0.717867,0.643481,0.615567
3,3,0.931533,4.0,1,1,16.7000,0.643481,0.628538,0.732342,0.936698,0.646289,0.643481,0.721144
4,1,0.931533,58.0,0,0,26.5500,0.643481,0.675159,0.950662,0.936698,0.592011,0.643481,0.615567
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1,0.931533,47.0,1,1,52.5542,0.643481,0.675159,0.950662,0.936698,0.725425,0.643481,0.721144
178,1,0.436317,33.0,0,0,5.0000,0.643481,0.675159,0.391155,0.391155,0.737957,0.643481,0.615567
179,1,0.931533,56.0,0,1,83.1583,0.737773,0.675159,0.950662,0.936698,0.592011,0.737773,0.721144
180,1,0.931533,19.0,0,0,30.0000,0.643481,0.675159,0.950662,0.936698,0.737957,0.643481,0.615567


ターゲットエンコーディングされた結果というのは、そのカテゴリをとる場合に目的変数が１である割合※を表しており、  
目的変数の情報を特徴量に組み込むことができる。  
このため、ターゲットエンコーディングをすると精度が上がることがある。  

※カテゴリ数が1である場合、平均をとると目的変数の値がそのまま入ってしまうのでスムージングという処理をしているので、  
　純粋な平均ではない。詳細は割愛。

### - 多クラス分類の場合

In [87]:
# データセットのカラム名やデータ型は分かっているので、dtypesを指定しておく。
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

In [88]:
# sexカラムの'.'を'others'で置き換え
df = df.with_columns(
    pl.when(pl.col('sex') == '.').then('others').otherwise(pl.col('sex')).alias('sex')
).to_pandas()

多クラス分類の場合、目的変数の各クラスをOne-Hotの形にして、  
そのそれぞれを目的変数としてターゲットエンコーディングを行う。  
３クラス分類であれば、カテゴリ変数一つに対して目的変数が3列あり、エンコーディング結果も3列になる。  

In [89]:
# 目的変数の全クラスをリストアップ
targets = df['species'].unique()
targets

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [90]:
# 多クラス分類の場合のTargetEncoding

for target in targets:
    # targetと等しいか否かをTrue・Falseで返すことにより、One-Hot的にする。
    # 平均の計算の際、Trueは1,Falseは0として計算される。
    target_y = (df['species'] == target)
    encoder = TargetEncoder()

    # 今回は'island'のカラムについてエンコーディングする。
    df['encoded_island_'+target] = encoder.fit_transform(df['island'], target_y)

In [91]:
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,encoded_island_Adelie,encoded_island_Chinstrap,encoded_island_Gentoo
0,Adelie,Torgersen,39.099998,18.700001,181.0,3750.0,MALE,0.978140,7.742061e-03,0.014118
1,Adelie,Torgersen,39.500000,17.400000,186.0,3800.0,FEMALE,0.978140,7.742061e-03,0.014118
2,Adelie,Torgersen,40.299999,18.000000,195.0,3250.0,FEMALE,0.978140,7.742061e-03,0.014118
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,None,0.978140,7.742061e-03,0.014118
4,Adelie,Torgersen,36.700001,19.299999,193.0,3450.0,FEMALE,0.978140,7.742061e-03,0.014118
...,...,...,...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,None,0.261905,7.385705e-08,0.738095
340,Gentoo,Biscoe,46.799999,14.300000,215.0,4850.0,FEMALE,0.261905,7.385705e-08,0.738095
341,Gentoo,Biscoe,50.400002,15.700000,222.0,5750.0,MALE,0.261905,7.385705e-08,0.738095
342,Gentoo,Biscoe,45.200001,14.800000,212.0,5200.0,FEMALE,0.261905,7.385705e-08,0.738095


上記のように元々のislandに対して３列のエンコーディング結果が対応する形となる。  
読み方としてはislandが'Torgersen'のとき、speciesがAdelieである割合は0.978, Chinstrapである割合はごくわずか、Gentooである割合は0.014、  
のようにざっくり読める。他のBiscoeでも同様。  
今回はislandのみエンコーディングしたが、他のカテゴリ変数に対しても同様に実施する必要あり。  

## 各encodingごとに精度比較

In [93]:
# データ読み込み
df = pl.from_pandas(sns.load_dataset('titanic'))
df = df.with_columns(
    pl.col('adult_male').cast(pl.Utf8),
    pl.col('alone').cast(pl.Utf8))

df.head()

survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
i64,i64,str,f64,i64,i64,f64,str,cat,str,str,cat,str,str,str
0,3,"""male""",22.0,1,0,7.25,"""S""","""Third""","""man""","""true""",null,"""Southampton""","""no""","""false"""
1,1,"""female""",38.0,1,0,71.2833,"""C""","""First""","""woman""","""false""","""C""","""Cherbourg""","""yes""","""false"""
1,3,"""female""",26.0,0,0,7.925,"""S""","""Third""","""woman""","""false""",null,"""Southampton""","""yes""","""true"""
1,1,"""female""",35.0,1,0,53.1,"""S""","""First""","""woman""","""false""","""C""","""Southampton""","""yes""","""false"""
0,3,"""male""",35.0,0,0,8.05,"""S""","""Third""","""man""","""true""",null,"""Southampton""","""no""","""true"""


In [106]:
# df.dtypesのユニークな値を取り出す
list(dict.fromkeys(df.dtypes))

[Int64, Utf8, Float64, Categorical]

In [107]:
# 量的変数、質的変数のカラム名をリストアップ
num_feat = df.select(pl.col([pl.Int64, pl.Float64])).drop('survived').columns
cat_feat = df.select(pl.col([pl.Utf8, pl.Categorical])).drop('alive').columns

In [109]:
df.describe()

describe,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
str,f64,f64,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str
"""count""",891.0,891.0,"""891""",891.0,891.0,891.0,891.0,"""891""","""891""","""891""","""891""","""891""","""891""","""891""","""891"""
"""null_count""",0.0,0.0,"""0""",177.0,0.0,0.0,0.0,"""2""","""0""","""0""","""0""","""688""","""2""","""0""","""0"""
"""mean""",0.383838,2.308642,null,29.699118,0.523008,0.381594,32.204208,null,null,null,null,null,null,null,null
"""std""",0.486592,0.836071,null,14.526497,1.102743,0.806057,49.693429,null,null,null,null,null,null,null,null
"""min""",0.0,1.0,"""female""",0.42,0.0,0.0,0.0,"""C""",null,"""child""","""false""",null,"""Cherbourg""","""no""","""false"""
"""max""",1.0,3.0,"""male""",80.0,8.0,6.0,512.3292,"""S""",null,"""woman""","""true""",null,"""Southampton""","""yes""","""true"""
"""median""",0.0,3.0,null,28.0,0.0,0.0,14.4542,null,null,null,null,null,null,null,null


fare(料金)の最小値がminというのは考えにくいので、  
チェックしてみる。

In [110]:
df.filter(pl.col('fare')==0.0)

survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
i64,i64,str,f64,i64,i64,f64,str,cat,str,str,cat,str,str,str
0,3,"""male""",36.0,0,0,0.0,"""S""","""Third""","""man""","""true""",null,"""Southampton""","""no""","""true"""
0,1,"""male""",40.0,0,0,0.0,"""S""","""First""","""man""","""true""","""B""","""Southampton""","""no""","""true"""
1,3,"""male""",25.0,0,0,0.0,"""S""","""Third""","""man""","""true""",null,"""Southampton""","""yes""","""true"""
0,2,"""male""",null,0,0,0.0,"""S""","""Second""","""man""","""true""",null,"""Southampton""","""no""","""true"""
0,3,"""male""",19.0,0,0,0.0,"""S""","""Third""","""man""","""true""",null,"""Southampton""","""no""","""true"""
0,2,"""male""",null,0,0,0.0,"""S""","""Second""","""man""","""true""",null,"""Southampton""","""no""","""true"""
0,2,"""male""",null,0,0,0.0,"""S""","""Second""","""man""","""true""",null,"""Southampton""","""no""","""true"""
0,2,"""male""",null,0,0,0.0,"""S""","""Second""","""man""","""true""",null,"""Southampton""","""no""","""true"""
0,3,"""male""",49.0,0,0,0.0,"""S""","""Third""","""man""","""true""",null,"""Southampton""","""no""","""true"""


In [94]:
# 特徴量からsurvived,aliveは落とす。
X = df.drop(['survived', 'alive']).to_pandas()
y = df['survived'].to_pandas()